# Семинар №5: Функциональное программирование и всякие разные приколы

<img src="images/mem_sem_05.png" width="500">

## 1. Генераторы  

Генераторы $-$ это такие итерируемые объекты-функции, которые выдают каждое значение ровно в тот момент, когда их это просят. Таким образом, это позволяет снизить нагрузку на память компьютера.   

**Пример:** одним из таких генераторов является `range()`, с которым мы уже не раз сталкивались. Чем отличается `range(10)` от списка $[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]$? А тем, что если бы мы итерировались по списку, то в памяти компьютера бы занялось 10 ячеек (своя ячейка для каждого числа), а при работе с генераторами ячейка занимается всего одна (сначала мы получили 0, потом его забыли и получили 1 и тд)

In [100]:
type(range(10))

range

Чтобы получить значения, генератор нужно распаковать:

In [101]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Пишем свой генератор:

In [103]:
def even_range(start, end):
    current = start
    while current < end:
        yield current # yield позволяет вернуть значение из функции, но не завершить саму функцию
        current += 2

In [104]:
for number in even_range(0, 10):
    print(number)

0
2
4
6
8


Значения в генераторе можно получать с помощью функции `next()`:

In [107]:
ranger = even_range(0, 10)

print(next(ranger))
print()

print(next(ranger))

0

2


__Пример:__ Числа Фибоначчи

In [108]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield a 
        a, b = b, a + b
        
for num in fibonacci(10):
    print(num)

1
1
2
3
5
8
13
21
34
55


## 2. list comprehensive  

Пишем простые циклы и всякие выражения в одну строку

In [97]:
even_list = [num for num in range(10) if num % 2 == 0]
print(even_list)

[0, 2, 4, 6, 8]


In [98]:
type(number ** 2 for number in range(5))

generator

## 3. Работа с файлами

Открывать файлы с помощью функции `open()` можно на чтение (`r`), запись (`w`), чтение и запись (`r+`) и на дозапись (`a`). Можем открывать файлы в бинарном виде, добавляя `b`, например, на чтение это будет `br`. Закрыть файл с помощью метода `close()`.

In [109]:
f = open('filename', 'w')
f.write('The world is changed \n I taste alcohol \n')

f.close()

In [111]:
f = open('filename', 'r+')
f.read()

'The world is changed \n I taste alcohol \n'

Читаем построчно

In [116]:
f.seek(0) # перемещаем указатель

print(f'Первая строка: {f.readline()}')
print(f'Вторая строка: {f.readline()}')

f.close()

Первая строка: The world is changed 

Вторая строка:  I taste alcohol 



**Контекстный менеджер** - используется, чтобы не париться о закрытии файла

In [117]:
with open('filename') as f:
    print(f.read())

The world is changed 
 I taste alcohol 



## 4. Исключения

###  try - except

Наивный способ:

In [3]:
try:
    1 / 0
except:
    print('Ошибка')

Ошибка


С указанием родительского класса (про классы подробнее на следующей паре):

In [ ]:
try:
    1 / 0
except Exception:
    print('Ошибка')

Не следует обрабатывать все исключения и оставлять блок `except` пустым. Чаще указывают конкретную ошибку:

In [5]:
while True:
    try:
        raw = input('Введите число: ')
        number = int(raw)
    except ValueError:
        print('Некорректное значение')
    else:
        break

Введите число: ff
Некорректное значение
Введите число: 3


Обработка нескольких исключений:

In [7]:
total_count = 100000

while True:
    try:
        raw = input('Введите число: ')
        number = int(raw)
        total_count = total_count / number
        break
    except (ValueError, ZeroDivisionError):
        print('Некорректное значение')

Введите число: ff
Некорректное значение
Введите число: 0
Некорректное значение
Введите число: 2


* `else` - что делать, если ошибки нет
* `finally` - выполняй всегда, несмотря на то, упал код или нет. Используется, когда нужно сделать что-то важное (прервать соединение / закрыть файл и тд) 

In [18]:
while True: 
    try:
        a = 1 
        b = int(input())
        a / b 
    except ZeroDivisionError:
        print('Exception!')
    else:
        print('No exceptions')
        break
    finally:
        print('This is the end!')

0
Exception!
This is the end!
3
No exceptions
This is the end!


### raise

In [30]:
try:
    1 / 0
except ZeroDivisionError as z:
    print(f'Got an exception "{z}"!')
    raise

Got an exception "division by zero"!


ZeroDivisionError: division by zero

In [28]:
a = None

if a is None:
    raise ValueError('a пустое')

ValueError: a пустое

### assert  

Используем только в тестах! НЕ в продакшн коде

In [125]:
assert True

In [126]:
# если генерируем с False, то AssertionError
assert 1 == 0

AssertionError: 

In [128]:
# можем добавлять сообщение
assert 1 == 0, '1 не равен 0'

AssertionError: 1 не равен 0

### Производительность  
С исключениями код выполняется дольше

In [129]:
%%timeit
my_dict = {'foo': 1}
for _ in range(1000):
    try:
        my_dict['bar']
    except KeyError:
        pass

232 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [130]:
%%timeit
my_dict = {'foo': 1}
for _ in range(1000):
    if 'bar' in my_dict:
        _ = my_dict['bar']

39 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


**Замечание:** В каждой библиотеке есть свои исключения. Они находятся в той же папке под именем `exceptions.py`

## 5. Приколы с функциями

Напомним, что функции - это такие же объекты, как и все остальное.   

Поэтому, например, мы можем **передавать в одну функцию в качестве аргумента другую функцию**

In [118]:
def caller(func, params):
    return func(*params)

def printer(name, origin):
    print(f"I\'m {name} of {origin}!")
    
caller(printer, ['Moana', 'Motunui'])

I'm Moana of Motunui!


Можем создавать **функции внутри функций**:

In [119]:
def get_multiplier():
    def inner(a, b):
        return a * b
    return inner

multiplier = get_multiplier()

multiplier(10, 11)

110

In [120]:
# в переменной multiplier лежит функция inner! НЕ get_multiplier
print(multiplier.__name__)

inner


**Концепция замыкания** - очень активно используется в декораторах.  

In [4]:
def get_multiplier(number):
    def inner(a):
        return a * number
    return inner

multiplier_by_2 = get_multiplier(2)
multiplier_by_2(20)

40

### type hinting  

по pep8 нужно явно указывать, что вы ожидаете передавать в функцию, что получать и само описание функции

In [41]:
def add(x: int, y: int) -> int:
    """Суммируем два числа"""
    return x + y

print(add(0, 1))

1


In [40]:
# с другими типами все равно работает - особенность питона
print(add('pep', '8'))

pep8


### \*args и \**kwargs

Штуки, которые позволяют передавать в функцию неопределенное кол-во параметров. **args** представляет собой `tuple`, а **kwargs** - `dict` 

In [89]:
def printer(*args):
    print(type(args))
    
    return [x**2 for x in args]

In [90]:
printer(1, 2, 3, 4, 5)

<class 'tuple'>


[1, 4, 9, 16, 25]

In [94]:
def printer(**kwargs):
    print(type(kwargs))
    
    for key, value in kwargs.items():
        print(f'{key} - {value}')

In [95]:
printer(a=10, b=11)

<class 'dict'>
a - 10
b - 11


In [96]:
arguments = {'user_id' : 131231, 'feedback' : 'good', 
            'description': {'age' : 22, 'sex': 'male'}}

printer(**arguments)

<class 'dict'>
user_id - 131231
feedback - good
description - {'age': 22, 'sex': 'male'}


### map и filter

`map` позволяет применять некую функцию к каждому элементу внутри какого-то итерабельного объекта

In [122]:
def squarify(a):
    return a ** 2

# нужно распаковать, так как это генератор
list(map(squarify, range(5)))

[0, 1, 4, 9, 16]

`filter` позволяет фильтровать какой-то итерабельный объект по предикаторной функции

In [124]:
def is_positive(a):
    return a > 0

# тоже распаковываем
list(filter(is_positive, range(-2, 3)))

[1, 2]

### Модуль `functools`

`reduce` позволяет сжимать данные, применяя последовательно функцию и запоминая результат

In [132]:
from functools import reduce

def multiply(a, b):
    return a * b

reduce(multiply, [1, 2, 3, 4, 5])

120

`partial` позволяет создавать новые функции, добавляя в какие-то аргументы значения по умолчанию

In [133]:
from functools import partial

def greeter(person, greeting):
    return '{}, {}!'.format(greeting, person)

hier = partial(greeter, greeting='Hi')
helloer = partial(greeter, greeting='Hello')

print(hier('brother'))
print(helloer('sir'))

Hi, brother!
Hello, sir!


## 6. Декораторы

Используем концепцию замыкания, но только 